In [2]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import pylab
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import librosa 
import pandas as pd
import glob
from keras.applications import VGG19, ResNet152
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Concatenate, LeakyReLU, Conv2D, MaxPooling2D,Input
from keras.utils import to_categorical
from tensorflow.keras import layers
from keras import optimizers
import keras
import torchaudio
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint
%matplotlib inline
from skimage.transform import resize
from random import sample


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
# Define a function to extract features from audio
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=22050)  # Load audio
    mel_feat = librosa.feature.melspectrogram(y=audio, sr=sr)
    power = librosa.power_to_db(mel_feat)
    power_resized = resize(power, (224, 224))
    return power_resized

In [8]:
# Path to your audio dataset
path = "/kaggle/input/environmental-sound-classification-50/audio/audio/44100"

# Lists to store extracted data and labels
labels = []
audio_data = []
# Iterate through audio files and extract features
for (root, dirs, files) in os.walk(path, topdown=True):
    for file in files:
        if file.endswith(".wav"):
            src = os.path.join(root, file)
            target = (file.split("-")[-1])[:-4]
            power_resized = extract_features(src)
            audio_data.append(np.stack([power_resized]*3, axis=-1))  # Convert to 3 channels
            labels.append(target)  # Extract label from file name

In [9]:
# Convert lists to numpy arrays
audio_data_array = np.array(audio_data)
labels_array = np.array(labels)

In [10]:
# Convert labels to categorical format
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels_array)
num_classes = len(label_encoder.classes_)
one_hot_labels = to_categorical(encoded_labels, num_classes=num_classes)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(audio_data_array, one_hot_labels, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Define your augmentation parameters
datagen = ImageDataGenerator(
    width_shift_range=0.2,          # Width shift by 20%
    height_shift_range=0.2,         # Height shift by 20%
    shear_range=0.2,                # Shear intensity
    zoom_range=-0.2,                 # Zoom by 20%
    horizontal_flip=True,           # Random horizontal flipping
    vertical_flip=True,             # Random vertical flipping
    brightness_range=[0.5, 1.5],    # Adjust brightness
    channel_shift_range=100.0,      # Channel shifting (RGB)
    fill_mode='nearest'             # Fill mode for new pixels
)

# Prepare an empty list to store generated images and labels
generated_data = []
generated_labels = []

# Define the number of images to generate for each class
images_per_class = 50

# Loop through each class
for label in range(y_train.shape[1]):  # Assuming y_train is one-hot encoded
    label_indices = np.where(y_train[:, label] == 1)[0]
    selected_indices = np.random.choice(label_indices, images_per_class, replace=False)
    
    # For each selected index, generate augmented images
    for idx in selected_indices:
        image = X_train[idx]  # Get the image for augmentation
        image = image.reshape((1,) + image.shape)  # Reshape for flow method
        
        # Flow method generates batches of augmented images
        augmented_images = datagen.flow(image, batch_size=1)
        
        # Iterate through the generated images
        for batch in augmented_images:
            generated_data.append(batch[0])  # Store augmented image
            generated_labels.append(label)  # Store corresponding label
            break  # Break the loop after one image to move to the next index

# Convert the generated data into numpy arrays
generated_data = np.array(generated_data)
generated_labels = np.array(generated_labels)

from tensorflow.keras.utils import to_categorical

# Convert generated_labels to one-hot encoded format
generated_labels_onehot = to_categorical(generated_labels, num_classes=num_classes)

# Merge generated images and labels with X_train and y_train
X_train_augmented = np.concatenate((X_train, generated_data), axis=0)
y_train_augmented = np.concatenate((y_train, generated_labels_onehot), axis=0)


In [12]:
audio_data_array.shape

(2000, 224, 224, 3)

In [3]:
# Load the data back
with open('/kaggle/input/dataesc/test_dataS_50.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

with open('/kaggle/input/dataesc/train_dataS_50.pkl', 'rb') as f:
    X_train_augmented, y_train_augmented = pickle.load(f)

with open('/kaggle/input/dataesc/val_dataS_50.pkl', 'rb') as f:
    X_val, y_val = pickle.load(f)
    

In [5]:
from tensorflow.keras.models import load_model
# Load the model
resnet152_model = load_model('/kaggle/input/resnet152-final/resnet152_modelS (2).h5')
vgg19_model = load_model('/kaggle/input/vgg19-and-effnet/vgg19_model_100SF.keras')
effnet_b0_model = load_model('/kaggle/input/vgg19-and-effnet/effnet_b0_model.keras')
ensemble_model = load_model('/kaggle/input/history-and-ensemble/ensemble_model.keras')

In [6]:
num_classes = 50

input_shape = (224, 224, 3)

In [3]:
# Load  VGG19 model 
vgg19 = VGG19(input_shape=input_shape, include_top=False, weights=None)
vgg19.load_weights('/kaggle/input/model-weights/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

vgg19_model = Sequential(name='custom_vgg19')

# Add VGG19 layers to the model
vgg19_model.add(vgg19)
vgg19_model.add(Dense(512, activation='relu', name='custom_dense_a'))
vgg19_model.add(Dropout(0.5, name='custom_drop1'))
vgg19_model.add(Dense(256, activation='relu', name='custom_dense_b'))
vgg19_model.add(Flatten(name='custom_flat'))
vgg19_model.add(Dense(224, activation='relu', name='custom_dense_c'))
vgg19_model.add(Dropout(0.5, name='custom_drop2'))
vgg19_model.add(Dense(num_classes, activation='softmax', name='custom_dense_d'))

# Freezing VGG19 layers
for layer in vgg19_model.layers[0].layers:
    layer.trainable = False

# Compiling the model
vgg19_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
vgg19_model.summary()

Model: "custom_vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 custom_dense_a (Dense)      (None, 7, 7, 512)         262656    
                                                                 
 custom_drop1 (Dropout)      (None, 7, 7, 512)         0         
                                                                 
 custom_dense_b (Dense)      (None, 7, 7, 256)         131328    
                                                                 
 custom_flat (Flatten)       (None, 12544)             0         
                                                                 
 custom_dense_c (Dense)      (None, 224)               2810080   
                                                                 
 custom_drop2 (Dropout)      (None, 224)              

In [ ]:
# Train your model using model.fit
history_vgg19 = vgg19_model.fit(X_train_augmented, y_train_augmented, batch_size=64,epochs=100, validation_data=(X_val, y_val))

In [7]:
loss, accuracy = vgg19_model.evaluate(X_test, y_test)

29/29 [==============================] - 10s 97ms/step - loss: 1.0377 - accuracy: 0.9467


In [8]:
loss, accuracy = vgg19_model.evaluate(X_val, y_val)

29/29 [==============================] - 2s 78ms/step - loss: 0.9798 - accuracy: 0.9556


In [4]:
resnet152 = ResNet152(weights=None, include_top=False, input_shape=input_shape)
resnet152.load_weights('/kaggle/input/model-weights/resnet152_weights_tf_dim_ordering_tf_kernels_notop.h5')
resnet152_model = Sequential()
resnet152_model.add(resnet152)
resnet152_model.add(layers.Dense(512, activation='relu'))
resnet152_model.add(layers.Dropout(0.5))
resnet152_model.add(layers.Dense(256, activation='relu'))
resnet152_model.add(layers.Flatten())
resnet152_model.add(layers.Dense(224, activation='relu'))
resnet152_model.add(layers.Dropout(0.5))
resnet152_model.add(layers.Dense(50, activation='softmax'))

for layer in resnet152_model.layers[0].layers:  
    layer.trainable = False

resnet152_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

resnet152_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 dense (Dense)               (None, 7, 7, 512)         1049088   
                                                                 
 dropout (Dropout)           (None, 7, 7, 512)         0         
                                                                 
 dense_1 (Dense)             (None, 7, 7, 256)         131328    
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense_2 (Dense)             (None, 224)               2810080   
                                                                 
 dropout_1 (Dropout)         (None, 224)               0

In [ ]:
history_resnet152 = resnet152_model.fit(X_train_augmented, y_train_augmented, batch_size=32,epochs=100, validation_data=(X_val, y_val))

In [9]:
loss, accuracy = resnet152_model.evaluate(X_test, y_test)

29/29 [==============================] - 8s 142ms/step - loss: 1.7329 - accuracy: 0.9600


In [10]:
loss, accuracy = resnet152_model.evaluate(X_val, y_val)

29/29 [==============================] - 4s 129ms/step - loss: 1.3390 - accuracy: 0.9556


In [5]:
def build_effnet_model(base_model):
    model = Sequential(name='custom_effnetb0')
    model.add(base_model)
    
    # Add custom layers on top of the base model
    model.add(Dense(512, activation='relu', name='custom_eff_Denselayer1'))
    model.add(Dropout(0.5, name='custom_eff_dropout1'))
    model.add(Dense(256, activation='relu', name='custom_eff_Denselayer2'))
    model.add(Flatten(name='custom_eff_flat'))
    model.add(Dense(224, activation='relu', name='custom_eff_Denselayer3'))
    model.add(Dropout(0.5, name='custom_eff_dropout2'))
    model.add(Dense(50, activation='softmax', name='custom_eff_Dense4'))
    
    # Set the base layers to non-trainable
    for layer in model.layers[0].layers:
        layer.trainable = False
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.00001), metrics=['accuracy'])
    
    return model

In [6]:
effnet_b0 = EfficientNetB0(input_shape=input_shape, include_top=False, weights=None)
effnet_b0.load_weights('/kaggle/input/model-weights/efficientnetb0_notop.h5') 

for layer in effnet_b0.layers:
    layer.trainable = False

    
effnet_b0_model = build_effnet_model(effnet_b0)

effnet_b0_model.summary()

Model: "custom_effnetb0"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 custom_eff_Denselayer1 (De  (None, 7, 7, 512)         655872    
 nse)                                                            
                                                                 
 custom_eff_dropout1 (Dropo  (None, 7, 7, 512)         0         
 ut)                                                             
                                                                 
 custom_eff_Denselayer2 (De  (None, 7, 7, 256)         131328    
 nse)                                                            
                                                                 
 custom_eff_flat (Flatten)   (None, 12544)         

In [ ]:
effnet_history = effnet_b0_model.fit(X_train_augmented, y_train_augmented, batch_size=64,epochs=100, validation_data=(X_val, y_val))

In [11]:
loss, accuracy = effnet_b0_model.evaluate(X_test, y_test)

29/29 [==============================] - 5s 58ms/step - loss: 0.5496 - accuracy: 0.9567


In [12]:
loss, accuracy = effnet_b0_model.evaluate(X_val, y_val)

29/29 [==============================] - 1s 45ms/step - loss: 0.4999 - accuracy: 0.9522


In [7]:
from tensorflow.keras.layers import GlobalAveragePooling2D, LSTM, Dense, Dropout, Reshape, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


# Load ResNet152 without top layers
resnet152 = ResNet152(weights=None, include_top=False, input_shape=(224, 224, 3))
resnet152.load_weights('/kaggle/input/model-weights/resnet152_weights_tf_dim_ordering_tf_kernels_notop.h5')

# Freeze layers in ResNet152
for layer in resnet152.layers:
    layer.trainable = False

effnet_b0 = EfficientNetB0(input_shape=input_shape, include_top=False, weights=None)
effnet_b0.load_weights('/kaggle/input/model-weights/efficientnetb0_notop.h5') 

# Freeze layers in EfficientNetB0
for layer in effnet_b0.layers:
    layer.trainable = False

# Input layer
input_layer = Input(shape=(224, 224, 3))

# Pass input through ResNet152 and EfficientNetB0 layers
x_resnet = resnet152(input_layer)
x_resnet = GlobalAveragePooling2D()(x_resnet)

x_effnet = effnet_b0(input_layer)
x_effnet = GlobalAveragePooling2D()(x_effnet)
# Concatenate ResNet152 and EfficientNetB0 outputs
x = Concatenate()([x_resnet, x_effnet])

    # Reshape to fit LSTM input requirements
x = Reshape((1, x.shape[1]))(x)

    # LSTM layer
x = LSTM(512)(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(224, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

    # Define the final model
ensemble_model = Model(inputs=input_layer, outputs=output)

ensemble_model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 resnet152 (Functional)      (None, 7, 7, 2048)           5837094   ['input_6[0][0]']             
                                                          4                                       
                                                                                                  
 efficientnetb0 (Functional  (None, 7, 7, 1280)           4049571   ['input_6[0][0]']             
 )                                                                                                
                                                                                              

In [1]:
# Train the ensemble model 
ensemble_history = ensemble_model.fit(X_train_augmented, y_train_augmented, epochs=100, batch_size=64, validation_data=(X_val, y_val))

In [13]:
loss, accuracy = ensemble_model.evaluate(X_test,y_test)

29/29 [==============================] - 13s 167ms/step - loss: 0.2069 - accuracy: 0.9678


In [14]:
loss, accuracy = ensemble_model.evaluate(X_val,y_val)

29/29 [==============================] - 5s 167ms/step - loss: 0.1438 - accuracy: 0.9733


**Accuracies on full original data are as below:**

In [ ]:
 loss, accuracy = vgg19_model.evaluate(audio_data_array, one_hot_labels)

In [ ]:
 loss, accuracy = resnet152_model.evaluate(audio_data_array, one_hot_labels)

In [ ]:
 loss, accuracy = effnet_b0_model.evaluate(audio_data_array, one_hot_labels)

In [17]:
 loss, accuracy = ensemble_model.evaluate(audio_data_array, one_hot_labels)

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

# Convert y_test to categorical labels if needed
y_test_categorical = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test

# Calculate predictions for each model
resnet152_preds = resnet152_model.predict(X_test)
vgg19_preds = vgg19_model.predict(X_test)
effnet_preds = effnet_b0_model.predict(X_test)
ensemble_preds = ensemble_model.predict(X_test)

# Calculate predictions for each model
# Ensure predictions are in the same format as categorical labels
resnet152_preds_categorical = np.argmax(resnet152_preds, axis=1)
vgg19_preds_categorical = np.argmax(vgg19_preds, axis=1)
effnet_preds_categorical = np.argmax(effnet_preds, axis=1)
ensemble_preds_categorical = np.argmax(ensemble_preds, axis=1)

# Accuracy for each model
resnet152_accuracy = accuracy_score(y_test_categorical, resnet152_preds_categorical)
vgg19_accuracy = accuracy_score(y_test_categorical, vgg19_preds_categorical)
effnet_accuracy = accuracy_score(y_test_categorical, effnet_preds_categorical)
ensemble_accuracy = accuracy_score(y_test_categorical, ensemble_preds_categorical)

# Precision, Recall, F1-score for each model
resnet152_precision = precision_score(y_test_categorical, resnet152_preds_categorical, average='weighted')
vgg19_precision = precision_score(y_test_categorical, vgg19_preds_categorical, average='weighted')
effnet_precision = precision_score(y_test_categorical, effnet_preds_categorical, average='weighted')
ensemble_precision = precision_score(y_test_categorical, ensemble_preds_categorical, average='weighted')


resnet152_recall = recall_score(y_test_categorical, resnet152_preds_categorical, average='weighted')
vgg19_recall = recall_score(y_test_categorical, vgg19_preds_categorical, average='weighted')
effnet_recall = recall_score(y_test_categorical, effnet_preds_categorical, average='weighted')
ensemble_recall = recall_score(y_test_categorical, ensemble_preds_categorical, average='weighted')

resnet152_f1 = f1_score(y_test_categorical, resnet152_preds_categorical, average='weighted')
vgg19_f1 = f1_score(y_test_categorical, vgg19_preds_categorical, average='weighted')
effnet_f1 = f1_score(y_test_categorical, effnet_preds_categorical, average='weighted')
ensemble_f1 = f1_score(y_test_categorical, ensemble_preds_categorical, average='weighted')

# Displaying the evaluation metrics
final_metrics = {
    'Model': ['ResNet152', 'VGG19', 'EfficientNet', 'Ensemble'],
    'Accuracy': [resnet152_accuracy, vgg19_accuracy, effnet_accuracy, ensemble_accuracy],
    'Precision': [resnet152_precision, vgg19_precision, effnet_precision, ensemble_precision],
    'Recall': [resnet152_recall, vgg19_recall, effnet_recall, ensemble_recall],
    'F1-Score': [resnet152_f1, vgg19_f1, effnet_f1, ensemble_f1]
}

final_metrics_df = pd.DataFrame(final_metrics)
print(final_metrics_df)


29/29 [==============================] - 10s 160ms/step
          Model  Accuracy  Precision    Recall  F1-Score
0     ResNet152  0.960000   0.965741  0.960000  0.960393
1         VGG19  0.946667   0.950992  0.946667  0.946990
2  EfficientNet  0.956667   0.960175  0.956667  0.956704
3      Ensemble  0.967778   0.970980  0.967778  0.967572
